<h1 align="center"><font color="yellow">LangChain 2: Chains de LLM usando GPT-3.5 e outros LLMs</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

# Contextualizando

Neste script sobre `LangChain`, exploraremos as cadeias (`Chains`), com foco nas cadeias `genéricas` e `utilitárias`, como a `LLMChain`. Esses são os principais recursos do `LangChain` que atuam como a base por trás de usos mais avançados do langchain, como `IA conversacional` (`chatbots`), `recuperação de ML aumentado` (retrieval augmented ML) e muito mais. 

LangChain é um Framework popular que permite aos usuários criar rapidamente aplicativos e pipelines em torno de Large Language Models. Ele se integra diretamente aos modelos `GPT-3` e `GPT-3.5` da OpenAI e às alternativas de código aberto do `Hugging Face`, como os modelos `flan-t5 do Google`. Ele pode ser usado para `chatbots`, `perguntas-respostas generativas` (GQA), `resumos` e muito mais. A ideia central da biblioteca é que podemos "encadear" diferentes componentes para criar casos de uso mais avançados em torno de LLMs. As cadeias podem consistir em vários componentes de vários módulos.

# Usando Chains

<font color="orange">As `Chains` são o núcleo do `LangChain`. Eles são simplesmente uma cadeia de componentes, executados em uma ordem específica.

A mais simples dessas cadeias é a `LLMChain`. Ele funciona recebendo a entrada de um usuário, passando para o primeiro elemento da cadeia — um `PromptTemplate` — para formatar a entrada em um prompt específico. O prompt formatado é então passado para o próximo (e último) elemento da cadeia — um `LLM`.

Começaremos importando todas as bibliotecas que usaremos neste exemplo.</font>

Antes disso vamos inserir a nossa Chave API OpenAI:

In [6]:
# Isto é quando usas o arquivo .env:
from dotenv import load_dotenv
import os
print('Carregando a minha chave Key: ', load_dotenv())
Eddy_API_KEY_OpenAI = os.environ['OPENAI_API_KEY'] 
Eddy_API_KEY_HuggingFace = os.environ["HUGGINGFACEHUB_API_TOKEN"]

Carregando a minha chave Key:  True


In [7]:
import inspect
import re

from getpass import getpass
from langchain import OpenAI, PromptTemplate
from langchain.chains import LLMChain, LLMMathChain, TransformChain, SequentialChain
from langchain.callbacks import get_openai_callback


In [8]:
llm = OpenAI(
    temperature=0, 
    openai_api_key=Eddy_API_KEY_OpenAI
            )

Um utilitário extra que usaremos é esta função que nos dirá quantos `Tokens` estamos usando em cada chamada. Esta é uma boa prática cada vez mais importante à medida que usamos ferramentas mais complexas que podem fazer várias chamadas à `API` (como `agentes`). `É muito importante ter um controle de quantos tokens estamos gastando para evitar gastos inesperados`.

In [10]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Gastou um total de {cb.total_tokens} tokens')

    return result


# O que são cadeias afinal?

As cadeias são um dos blocos de construção fundamentais desta biblioteca (como você pode imaginar!).

A definição oficial de `Chains` é a seguinte:


<font color="pink">Uma cadeia (chain) é composta de elos, que podem ser primitivos ou outras cadeias. Primitivos podem ser `prompts`, `llms`, `utils` ou outras `cadeias`.</font>


Portanto, uma cadeia é basicamente um `pipeline` que processa uma entrada usando uma combinação específica de primitivas. Intuitivamente, pode ser pensado como uma `'etapa'` que executa um determinado conjunto de operações em uma entrada e retorna o resultado. Eles podem ser qualquer coisa, desde uma passagem baseada em `prompt` por meio de um `LLM` até a aplicação de uma função `Python` a um texto.

As cadeias são divididas em três tipos: `cadeias utilitárias`, `cadeias genéricas` e `cadeias de combinar documentos`. Neste script, vamos nos concentrar nos *dois primeiros*, já que o terceiro é muito específico (será abordado oportunamente).

* <font color="red">Cadeias utilitárias:</font> cadeias que geralmente são usadas para extrair uma resposta específica de um LLM com um propósito muito restrito e estão prontas para serem usadas fora da caixa.

* <font color="red">Cadeias Genéricas:</font> cadeias que são usadas como blocos de construção para outras chains, mas não podem ser usadas fora da caixa por conta própria.


Vamos dar uma olhada no que essas redes têm a oferecer!


# <font color="red">Cadeias de utilitárias</font> (`Utility Chains`)


Vamos começar com uma cadeia de utilidades simples. A `LLMMathChain` dá aos LLMs a capacidade de fazer matemática. Vamos ver como isso funciona!

`Dica profissional:` use `verbose=True` para ver quais são as diferentes etapas da cadeia!

In [11]:
llm_math = LLMMathChain(
    llm=llm,
    verbose=True
                       )


count_tokens(llm_math, "Quanto é 13 elevado à potência de 0.3432?")




> Entering new LLMMathChain chain...
Quanto é 13 elevado à potência de 0.3432?
```python
print(13**0.3432)
```

Answer: 2.4116004626599237

> Finished chain.
Gastou um total de 271 tokens


'Answer: 2.4116004626599237\n'

<font color="orange">Vamos ver o que está acontecendo aqui. A rede recebeu uma pergunta em linguagem natural e a enviou ao LLM. O LLM retornou um código `Python` que a cadeia compilou para nos dar uma resposta. Algumas perguntas surgem. Como o llm sabia que queríamos que ele retornasse o código `Python`?</font>


## Enter Prompts

A pergunta que enviamos como entrada para a cadeia não é a única entrada que o llm recebe 😉. O `Input` é inserida em um contexto mais amplo, que fornece instruções precisas sobre como interpretar a entrada que enviamos. `Isso é chamado de prompt`. Vamos ver qual é o prompt dessa cadeia!

In [14]:
print(llm_math.prompt.template)

You are GPT-3, and you can't do math.

You can do basic math, and your memorization abilities are impressive, but you can't do any complex calculations that a human could not do in their head. You also have an annoying tendency to just make up highly specific, but wrong, answers.

So we hooked you up to a Python 3 kernel, and now you can execute code. If anyone gives you a hard math problem, just use this format and we’ll take care of the rest:

Question: ${{Question with hard calculation.}}
```python
${{Code that prints what you need to know}}
```
```output
${{Output of your code}}
```
Answer: ${{Answer}}

Otherwise, use this simpler format:

Question: ${{Question without hard calculation}}
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?

```python
print(37593 * 67)
```
```output
2518731
```
Answer: 2518731

Question: {question}



Ok .. vamos ver o que temos aqui. Portanto, estamos literalmente dizendo ao LLM que, para problemas matemáticos complexos, ele não deve tentar fazer matemática sozinho, mas sim imprimir um código `Python` que calculará o problema matemático. Provavelmente, se apenas enviássemos a query sem nenhum contexto, o LLM tentaria (`e falharia`) calcular isso por conta própria. 

`Espere! Isso é testável.. vamos experimentar!` 🧐

In [16]:
# Definimos o prompt para ter apenas a pergunta que fazemos
prompt = PromptTemplate(
    input_variables=['question'],
    template='{question}'
                       )


llm_chain = LLMChain(prompt=prompt, llm=llm)

# Pedimos ao LLM a resposta sem contexto:
print(count_tokens(llm_chain, "Quanto é 13 elevado à potência de 0.3432?"))

Gastou um total de 35 tokens


A resposta é aproximadamente 2,068.


`Resposta errada!`

Aqui reside o poder do `prompting` e um dos nossos insights mais importantes até agora:

`Insight:` ao usar prompts de forma inteligente, podemos `forçar o LLM` a evitar armadilhas comuns, programando-o explícita e intencionalmente para se comportar de uma determinada maneira.


Outro ponto interessante sobre essa cadeia é que ela não apenas executa uma entrada por meio do LLM, mas também compila o código Python posteriormente. Vamos ver exatamente como isso funciona.

In [17]:
print(inspect.getsource(llm_math._call))

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        llm_executor = LLMChain(
            prompt=self.prompt, llm=self.llm, callback_manager=self.callback_manager
        )
        self.callback_manager.on_text(inputs[self.input_key], verbose=self.verbose)
        t = llm_executor.predict(question=inputs[self.input_key], stop=["```output"])
        return self._process_llm_result(t)



Portanto, podemos ver aqui que, se o LLM retornar o código Python, iremos compilá-lo com um simulador `Python REPL*`. Agora temos a imagem completa da cadeia: ou o LLM retorna uma resposta (`para problemas matemáticos simples`) ou retorna o `código Python` que compilamos para obter uma resposta exata para problemas mais difíceis. `Inteligente!`


Observe também que aqui temos nosso primeiro exemplo de `composição de cadeia`, um conceito-chave por trás do que torna o `LangChain` especial. Estamos usando o `LLMMathChain` que, por sua vez, inicializa e usa um `LLMChain` (uma `'cadeia genérica'`) quando chamado. Podemos fazer qualquer número arbitrário dessas composições, efetivamente `'encadeando'` muitas dessas cadeias para obter um comportamento altamente complexo e personalizável.


As <font color="yellow">cadeias de utilitários</font> geralmente seguem essa mesma estrutura básica: há um `prompt` para restringir o LLM a retornar um tipo muito específico de resposta de uma determinada query. Podemos pedir ao LLM para criar `queries SQL`, chamadas de API e até mesmo criar `comandos Bash` em tempo real 🔥


A lista continua a crescer à medida que o `LangChain` se torna cada vez mais flexível e poderoso, então encorajamos você a dar uma olhada e mexer mais nos [Notebooks](https://python.langchain.com/en/latest/gallery.html?highlight=notebooks#misc-colab-notebooks) e ver se você pode achar eles interessantes.

* Um `Python REPL` (Read-Eval-Print Loop) é um shell interativo para executar o código Python linha por linha

# <font color="red">Cadeias genéricas</font> (`Generic Chains`)


Existem apenas `três Cadeias Genéricas` no `LangChain` e iremos mostrá-las todas no mesmo exemplo. Vamos!

Digamos que tivemos a experiência de obter textos de entrada sujos. Especificamente, como sabemos, os LLMs nos cobram pelo número de `Tokens` que usamos e não ficamos felizes em pagar a mais quando a entrada tem caracteres extras. Além disso, não é legal 😉

`Primeiro`, construiremos uma função de transformação personalizada para limpar o espaçamento de nossos textos. Em seguida, usaremos essa função para construir uma cadeia onde inserimos nosso texto e esperamos um texto limpo como saída.

In [18]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    
    # Substituímos várias novas linhas e vários espaços por um único
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}


É importante ressaltar que, quando inicializamos a cadeia, não enviamos um `LLM` como argumento. Como você pode imaginar, não ter um `LLM` torna as habilidades dessa cadeia muito mais fracas do que no exemplo que vimos anteriormente. No entanto, como veremos a seguir, combinar essa cadeia com outras cadeias pode nos dar resultados altamente desejáveis.

In [19]:
clean_extra_spaces_chain = TransformChain(input_variables=["text"],
                                          output_variables=["output_text"],
                                          transform=transform_func
                                         )


In [24]:
# Vejamos como funciona:
print(clean_extra_spaces_chain.run('Um texto aleatório   com    algum espaçamento irregular.\n\n\n    Outro aqui    também.'))


Um texto aleatório com algum espaçamento irregular.
 Outro aqui também.


<font color="orange">Ótimo! Agora as coisas vão ficar interessantes.

Digamos que queremos usar nossa cadeia para limpar um texto de entrada e, em seguida, parafrasear a entrada em um estilo específico, digamos um poeta ou um policial. Como sabemos agora, o `TransformChain` <font color="red">não usa um LLM</font>, então o estilo terá que ser feito em outro lugar. É aí que entra o nosso `LLMChain`. Já sabemos sobre essa cadeia e sabemos que podemos fazer coisas legais com `prompts inteligentes`, então vamos arriscar!


`Primeiro` vamos construir o `Template Prompt`:</font>

In [25]:
template = """Parafraseie este texto:

{output_text}

No estilo de um {style}.

Paraphrase: """


prompt = PromptTemplate(input_variables=["style", "output_text"],
                        template=template
                       )


E a seguir, inicializamos nossa Chain:

In [26]:
style_paraphrase_chain = LLMChain(llm=llm,
                                  prompt=prompt,
                                  output_key='final_output'
                                 )


Ótimo! Observe que o texto de entrada no `Template` é chamado `'output_text'`. Você consegue adivinhar por quê?

Vamos passar a saída do `TransformChain` para o `LLMChain`!

Finalmente, precisamos combiná-los para funcionar como uma cadeia integrada. Para isso, usaremos `SequentialChain`, que é nosso terceiro bloco de construção de `cadeia genérica`.

In [27]:
sequential_chain = SequentialChain(chains=[clean_extra_spaces_chain, style_paraphrase_chain],
                                   input_variables=['text', 'style'],
                                   output_variables=['final_output']
                                  )



Nosso input é a descrição dos documentos LangChain de quais chains estão sujas com alguns espaços extras ao redor.

In [28]:
input_text = """
As cadeias nos permitem combinar vários


componentes juntos para criar um aplicativo único e coerente.

Por exemplo, podemos criar uma cadeia que recebe entrada do usuário,    formatá-la com um PromptTemplate,

e então passa a resposta formatada para um LLM. Podemos construir cadeias mais complexas combinando     várias cadeias juntas ou


combinando cadeias com outros componentes.
"""

Estamos prontos. Hora de ser criativo!

In [30]:
count_tokens(sequential_chain,
             {'text': input_text, 'style': 'Um rapper dos anos 90'}
            )


Gastou um total de 419 tokens


'\nAs cadeias nos dão a habilidade de juntar vários elementos e criar um aplicativo único e consistente. Por exemplo, podemos montar uma cadeia que pegue a entrada do usuário, a formate com um PromptTemplate, e então passe a resposta formatada para um LLM. Nós podemos construir cadeias mais complexas juntando várias cadeias ou combinando cadeias com outros componentes.\n\nNo estilo de um Um rapper dos anos 90.\n\nParaphrase:\nNós temos a capacidade de juntar vários elementos e criar um aplicativo único e sólido. Por exemplo, podemos montar uma cadeia que pegue a entrada do usuário, a formate com um PromptTemplate, e então passe a resposta formatada para um LLM. Nós podemos construir cadeias mais complexas j'

# Uma nota sobre `LangChain-hub`

`LangChain-hub` é uma biblioteca irmã do `LangChain`, onde todas as `chains`, `agentes` e `prompts` são serializados para nosso uso.

In [31]:
from langchain.chains import load_chain


Carregar do `langchain-hub` é tão fácil quanto encontrar a cadeia que você deseja carregar no repositório e, em seguida, usar `load_chain` com o caminho (path) correspondente. Também temos `load_prompt` e `initialize_agent`, mas falaremos mais sobre isso depois. Vamos ver como podemos fazer isso com nosso `LLMMathChain` que vimos anteriormente:

In [ ]:
llm_math_chain = load_chain('lc://chains/llm-math/chain.json')

<font color="red">E se quisermos alterar alguns dos parâmetros de configuração?</font> Podemos simplesmente substituí-lo após o carregamento: